# E3. 基于 Bert 和 fastNLP 来实现情感分类

&emsp;&emsp;本篇教程将为您详细展示如何使用 `fastNLP` 和`Bert`来实现简单的情感任务。

&emsp;&emsp;本篇教程的数据集是SST-2 文本情感二分类数据集。


## 1. 准备工作：加载数据，加载 tokenizer、预处理 dataset、dataloader

&emsp;&emsp;在此教程中，我们仍旧使用 `sst-2` 来训练模型，实现情感分类。首先使用 `datasets` 来加载 `sst-2`。



In [1]:
from fastNLP.io import DataBundle
from fastNLP import DataSet
from paddlenlp.datasets import load_dataset

train_dataset, val_dataset, test_dataset = load_dataset("glue","sst-2", splits=["train", "dev", "test"])

print("训练集大小：", len(train_dataset))
for i in range(3):
    print(train_dataset[i])

训练集大小： 67349
{'sentence': 'hide new secretions from the parental units ', 'labels': 0}
{'sentence': 'contains no wit , only labored gags ', 'labels': 0}
{'sentence': 'that loves its characters and communicates something rather beautiful about human nature ', 'labels': 1}


&emsp;&emsp;可以看到，原本的数据集仅包含英文的文本和标签，这样的数据是无法被模型识别的，我们需要使用 tokenizer 对文本进行分词并转换为数字形式的结果。

&emsp;&emsp;我们加载已经预训练好的中文分词模型 `bert-base-uncased`，将分词的过程写在函数 _process 中，然后调用数据集的 map 函数对每一条数据进行分词。其中：

- 参数 `max_length` 代表句子的最大长度；
- `padding="max_length"` 表示将长度不足的结果 padding 至和最大长度相同；
- `truncation=True` 表示将长度过长的句子进行截断。

&emsp;&emsp;至此，我们得到了每条数据长度均相同的数据集。

In [ ]:
from paddlenlp.transformers import AutoTokenizer

max_len = 32
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
def _process(data):
    data.update(tokenizer(
        data["sentence"],
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
    ))
    return data

train_dataset.map(_process, num_workers=5)
val_dataset.map(_process, num_workers=5)
test_dataset.map(_process, num_workers=5)

In [3]:
print(train_dataset[0])

{'sentence': 'hide new secretions from the parental units ', 'labels': 0, 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


&emsp;&emsp;得到数据集之后，我们便可以将数据集包裹在 `PaddleDataLoader` 中，用于之后的训练。

&emsp;&emsp;`fastNLP` 提供的 `PaddleDataLoader` 拓展了 `paddle.io.DataLoader` 的功能，详情可以查看相关的文档。

In [7]:
from fastNLP.core import PaddleDataLoader
import paddle.nn as nn

train_dataloader = PaddleDataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = PaddleDataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = PaddleDataLoader(test_dataset, batch_size=1, shuffle=False)

## 3. 模型训练：加载 BERT、fastNLP 参数匹配、fine-tuning

&emsp;&emsp;为了实现文本分类，我们首先需要定义文本分类的模型。

&emsp;&emsp;`paddlenlp.transformers` 提供了模型 `AutoModelForSequenceClassification`，我们可以利用它来加载不同权重的文本分类模型。

&emsp;&emsp;在 `fastNLP` 中，我们可以定义 `train_step` 和 `evaluate_step` 函数来实现训练和验证过程中的不同行为。

&emsp;&emsp;`train_step` 函数在获得返回值 `logits` （大小为 (batch_size, num_labels)）后计算交叉熵损失 `CrossEntropyLoss`，然后将 `loss` 放在字典中返回。`fastNLP` 也支持返回 `dataclass` 类型的训练结果，但二者都需要包含名为 loss 的键或成员。

&emsp;&emsp;`evaluate_step` 函数在获得返回值 `logits` 后，将 `logits` 和标签 `label` 放在字典中返回。

&emsp;&emsp;这两个函数的参数均为数据集中字典键的子集，`fastNLP` 会自动进行参数匹配然后输入到模型中

In [ ]:
from paddlenlp.transformers import AutoModelForSequenceClassification
import paddle.nn as nn

class SeqClsModel(nn.Layer):
    def __init__(self, model_checkpoint, num_labels):
        super(SeqClsModel, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_checkpoint,
            num_classes=num_labels,
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        logits = self.model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        return logits

    def train_step(self, input_ids, attention_mask, token_type_ids, labels):
        logits = self(input_ids, attention_mask, token_type_ids)
        loss = nn.CrossEntropyLoss()(logits, labels)
        return {"loss": loss}

    def evaluate_step(self, input_ids, attention_mask, token_type_ids, labels):
        logits = self(input_ids, attention_mask, token_type_ids)
        return {'pred': logits, 'target': labels}

model = SeqClsModel(model_checkpoint, num_labels=2)

&emsp;&emsp;初始化优化器 `optimizer`、训练模块 `Trainer`，最后，使用之前完成的`train_dataloader` 和`evaluate_dataloader`，训练模块 `Trainer`，得到训练结果。

In [9]:
import paddle
from fastNLP import LRSchedCallback, LoadBestModelCallback
from fastNLP import Trainer, Accuracy
from paddlenlp.transformers import LinearDecayWithWarmup

n_epochs = 10
num_training_steps = len(train_dataloader) * n_epochs
optimizer = paddle.optimizer.AdamW(
    learning_rate=5e-5,
    parameters=model.parameters(),
)
trainer = Trainer(
    model=model,
    driver="paddle",
    optimizers=optimizer,
    device=1,
    n_epochs=n_epochs,
    train_dataloader=train_dataloader,
    evaluate_dataloaders=val_dataloader,
    metrics={"accuracy": Accuracy()}
)


In [10]:
trainer.run()

[09:47:01] INFO     Running evaluator sanity check for 2 batches.              ]8;id=385044;file:///remote-home/kychen/anaconda3/envs/paddlepaddle/lib/python3.8/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=647938;file:///remote-home/kychen/anaconda3/envs/paddlepaddle/lib/python3.8/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Output()

Output()

----------------------------- Eval. results on Epoch:1, Batch:0 -----------------------------

{
  "acc#accuracy": 0.81
}

...

{
  "acc#accuracy": 0.87
}

----------------------------- Eval. results on Epoch:9, Batch:0 -----------------------------

{
  "acc#accuracy": 0.89
}

---------------------------- Eval. results on Epoch:10, Batch:0 -----------------------------

{
  "acc#accuracy": 0.86
}

In [ ]:
trainer.evaluator.run()

Output()

{'acc#accuracy': 0.86}